In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import np_utils
from keras.preprocessing import image
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline

In [ ]:
trainDataFrame = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
testDataFrame = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

trainDataFrame['ClassId'] = trainDataFrame['ClassId'].astype(str)
testDataFrame['ClassId'] = testDataFrame['ClassId'].astype(str)

for i in range(0, len(trainDataFrame['ClassId'])):
    if len(trainDataFrame['ClassId'][i]) == 1:
        trainDataFrame['ClassId'][i] = '0' + trainDataFrame['ClassId'][i]
for i in range(0, len(testDataFrame['ClassId'])):
    if len(testDataFrame['ClassId'][i]) == 1:
        testDataFrame['ClassId'][i] = '0' + testDataFrame['ClassId'][i]

print(trainDataFrame['ClassId'][20], trainDataFrame['Path'][20])
img = Image.open('../input/gtsrb-german-traffic-sign/' + trainDataFrame['Path'][20])
plt.imshow(img)

In [ ]:
trainDataGen = image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2
)
testDataGen = image.ImageDataGenerator(
    rescale=1./255
)

trainDataGenerator = trainDataGen.flow_from_dataframe(
    dataframe=trainDataFrame,
    directory='../input/gtsrb-german-traffic-sign/',
    x_col='Path',
    y_col='ClassId',
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical'
)
testDataGenerator = testDataGen.flow_from_dataframe(
    dataframe=testDataFrame,
    directory='../input/gtsrb-german-traffic-sign/',
    x_col='Path',
    y_col='ClassId',
    target_size=(32, 32),
    batch_size=16,
    class_mode='categorical'
)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
print(model.summary())

In [ ]:
history = model.fit(trainDataGenerator, epochs=5, verbose=1)

In [ ]:
loss = history.history['loss']
# val_loss = history.history['val_loss']
acc = history.history['accuracy']
# val_acc = history.history['val_acc']
epoches = range(0, len(loss))
plt.plot(epoches, loss, 'b', label='training loss')
# plt.plot(epoches, val_loss, 'b-', label='validation loss')
plt.legend()
plt.figure()
plt.plot(epoches, acc, 'g', label='training accuracy')
# plt.plot(epoches, val_acc, 'g-', label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
model.evaluate(testDataGenerator, verbose=1)

In [ ]:
test_batch = 1
plt.figure(figsize=(64, 64))
plt.tight_layout()
for i in range(0, 16):
    plt.subplot(16, 2, 2 * i + 1), plt.axis('off')
    plt.imshow(testDataGenerator[test_batch][0][i])
    pre = model.predict_classes(testDataGenerator[test_batch][0])
    img = Image.open('../input/gtsrb-german-traffic-sign/Meta/' + str(pre[i]) + '.png')
    plt.subplot(16, 2, 2 * i + 2), plt.axis('off')
    plt.imshow(img)